In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

from var import DATA_IN, DATA_OUT

## TIDs catalog

In [ ]:
df_tid = pd.read_csv(
    Path(DATA_IN, 'TID_catalog.csv'),
    header=0,
    names=[
        'duration',
        'period',
        'amplitude',
        'spectral_contribution',
        'velocity',
        'azimuth',
        'quality_index',
        'datetime',
    ],
)

df_tid['datetime'] = pd.to_datetime(df_tid['datetime'], format='%d-%b-%Y %H:%M:%S')

In [ ]:
df_tid.sample(3)

## Solar Data

In [ ]:
df_solar = pd.read_csv(
    Path(DATA_IN, 'solar_data.csv'),
    header=0,
    names=[
        'day_of_rotation',
        'n_sunsposts',
        'f_107_adj',
        'date',
    ],
)

df_solar['date'] = pd.to_datetime(df_solar['date'], format='%d-%b-%Y')

In [ ]:
df_solar.sample(3)